$
\newcommand{\ra}{\vec{r_1}}
\newcommand{\rb}{\vec{r_2}}
\newcommand{\rab}{\vec{r_{12}}}
\newcommand{\d}{\mathrm{d}}
\newcommand{\g}{\vec{g}}
$

Gravity exerts the following forces on the masses at $\ra$ and $\rb$:

$$
\begin{align}
\vec{F_{G1}} &= \begin{pmatrix} 0\\ - g m_1 \end{pmatrix} \\
\vec{F_{G2}} &= \begin{pmatrix} 0\\ - g m_2 \end{pmatrix}
\end{align}
$$

The position of the masses are subject to the following constraints:

$$
\begin{align}
|\ra| &\leq R_1 \\
|\rb| &\leq R_2 \\
\rab &\overset{\text{rhs}}{=} \rb - \ra \\
|\rab| &= R_{12}
\end{align}
$$

For given $R_1$, $R_2$ and $R_{12}$.

Let's rhsine the symbols encountered so far in `sympy`:

In [7]:
import sympy
piecewise = sympy.functions.elementary.piecewise
Eq = sympy.Eq

sympy.init_printing()

r = sympy.Matrix(2, 2, sympy.symbols('r_{1:3}^{1:3}', real=True, nonnegative=True))
m = sympy.Matrix(1, 2, sympy.symbols('m_{1:3}', real=True, nonnegative=True))

R = sympy.Matrix(1, 2, sympy.symbols('R_{1:3}', real=True, nonnegative=True))
g = sympy.symbols('g', real=True, positive=True)
R12 = sympy.symbols('R_{12}', real=True, nonnegative=True)

l = sympy.Matrix(1, 2, sympy.symbols('l_{1:3}', real=True, nonnegative=True))
l12 = sympy.symbols('l_{12}', real=True, nonnegative=True)
r12 = r[1,:] - r[0,:]

FG = sympy.Matrix([0, 1])*-m*g

The above constraint on $|\ra|$ corresponds to a constraint force which we'll call $\vec{F_{c1}}$ with

$$
\begin{align}
\vec{F_{c1}} &= \vec{0},\,\text{if $|\ra| \leq R_1$} \\
\vec{F_{c1}} &= - \left(\vec{F_{c12,1}} + \vec{F_{G1}} \right) \cdot \ra \frac{\ra}{r_1},\,\text{if $|\ra| \gt R_1$}
\end{align}
$$

and analogously for $\vec{F_{c2}}$. Let's rhsine them in `sympy`:



In [8]:
Fc = sympy.Matrix(2, 2, sympy.symbols('F_{1:3\,1:3}^{(c)}'))
Fc12 = sympy.Matrix(2, 2, sympy.symbols('F_{1:3\,1:3}^{(c12)}'))

In [9]:
def create_Fc_rhs(i, j):
    Ri = R[i]
    ri = r[i,:]
    li = l[i]
    
    sign = -1 if i == 0 else 1
    
    Fc_rhs_gt = sign*(Fc12[i,:] + FG[i,:]).dot(ri)*ri.normalized()
    
    Fc_rhs = piecewise.Piecewise(piecewise.ExprCondPair(0, li <= Ri),
                                 piecewise.ExprCondPair(Fc_rhs_gt[j], li > Ri))
    return Fc_rhs

The two forces $\vec{F_{c12,1}}$ and $\vec{F_{c12,2}}$ above correspond to the constraint on $|\rab|$ at $\ra$ and $\rb$, respectively. They are rhsined by:

$$
\begin{align}
\vec{F_{c12,1}} &= \vec{0},\,\text{if $|\rab| = R_{12}$} \\
\vec{F_{c12,1}} &= \left(\vec{F_{c1}} + \vec{F_{G1}} \right) \cdot \rab \frac{\rab}{r_{12}},\,\text{if $|\rab| \gt R_{12}$} \\
\vec{F_{c12,1}} &= - \left(\vec{F_{c1}} + \vec{F_{G1}} \right) \cdot \rab \frac{\rab}{r_{12}},\,\text{if $|\rab| \lt R_{12}$}
\end{align}
$$

and, again, analogously for $\vec{F_{c12,2}}$. In `sympy`:


In [10]:
def create_Fc12_rhs(i, j):
    sign = 1 if i == 0 else -1
    
    Fc12_rhs_gt = sign*(Fc[i,:]+FG[i,:]).dot(r12)*r12.normalized()
    Fc12_rhs_lt = -Fc12_rhs_gt
    
    Fc12_rhs = piecewise.Piecewise(piecewise.ExprCondPair(Fc12_rhs_lt[j], l12 < R12),
                                   piecewise.ExprCondPair(0, Eq(l12, R12)),
                                   piecewise.ExprCondPair(Fc12_rhs_gt[j], l12 > R12))
    return Fc12_rhs

Finally, let's define the total resulting force acting at $\ra$ (and analogously for $\rb$):

$$\vec{F_1} = \vec{F_{G1}} + \vec{F_{c1}} + \vec{F_{c12,1}}$$

The corresponding `sympy` code looks like this:

In [11]:
F = FG + Fc + Fc12

We've not got all the required pieces to tie the knot and rhsine our system of equations for the static case of $\vec{F_l}=0$ for $l=1,\,2$:

In [29]:
def create_F_eqs(i, j):
    return [Eq(F[i,j], 0), Eq(Fc[i,j], create_Fc_rhs(i,j)), Eq(Fc12[i,j], create_Fc12_rhs(i,j))]

F_eqs = [eq for i in range(2) for j in range(2) for eq in create_F_eqs(i, j)]
l_eqs = [Eq(l[0], r[0,:].norm()), Eq(l[1], r[1,:].norm()), Eq(l12, r12.norm())]
eqs = F_eqs + l_eqs
master_eq = sympy.And(*eqs)
master_eq_folded = sympy.piecewise_fold(master_eq)
master_eq_folded

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                   

In [34]:
type(master_eq_folded.args[-1].expr)

And

In [14]:
from IPython.display import display, Math

for eq in eqs:
    display(Math(sympy.latex(eq)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [28]:
wrt = list(r.reshape(1,4))
sympy.nonlinsolve([master_eq_folded], wrt)

TypeError: expecting bool or Boolean, not `F_{1,1}^{(c)} + F_{1,1}^{(c12)}`.

In [128]:
r11.is_complex

In [17]:
x, y, s = sympy.symbols('x y s')
pw = piecewise.Piecewise(piecewise.ExprCondPair(x - y, s <= 1), piecewise.ExprCondPair(2*x - y, True))
sympy.And

In [21]:
sympy.nonlinsolve([pw, sympy.sqrt(x**2+y**2) - s], [x])